In [1]:
from omnidata.framework import get_builder
from omnilearn import util

No profile path provided (can be set with environment variable: FIG_PROFILE)
2022-09-01 16:41:43.543708: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /is/ei/fleeb/miniconda3/lib/python3.8/site-packages/cv2/../../lib64:
2022-09-01 16:41:43.543723: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
opt = get_builder('optim')
opt

omnilearn.novo.optim.PytorchOptimizer

In [3]:
hparams = list(opt.plan('adam'))
hparams

[('lr', <omnilearn.novo.optim.Optimizer.Hyperparameter at 0x7ff7182be1c0>),
 ('beta1', <omnilearn.novo.optim.Optimizer.Hyperparameter at 0x7ff7182be220>),
 ('beta2', <omnilearn.novo.optim.Optimizer.Hyperparameter at 0x7ff7182be280>),
 ('eps', <omnilearn.novo.optim.Optimizer.Hyperparameter at 0x7ff7182be2e0>),
 ('weight_decay',
  <omnilearn.novo.optim.Optimizer.Hyperparameter at 0x7ff7182be340>),
 ('ident', <omnilearn.novo.optim.Optimizer.Hyperparameter at 0x7ff7182be400>),
 ('amsgrad',
  <omnilearn.novo.optim.Optimizer.Hyperparameter at 0x7ff7182be3a0>)]

In [4]:
lrp = hparams[0][1]

In [5]:
optim = opt.build()
optim

Adam ()

In [6]:
optim.lr

0.001

In [ ]:
list(optim.full_spec())

In [ ]:
from src import main

In [ ]:
list(main.Supervised_Model.full_spec())

In [ ]:
'asd'.split('.')

In [1]:
from omnibelt import nodes

In [2]:
Node = nodes.AutoTreeNode

In [3]:
g = Node.from_raw({'a': [12, 13], 'b': {'x': 1, 'y': 2}})

In [4]:
g.set('b.z', 100)

In [5]:
g.set('a.6', 20)
g.set('a.3', [])

In [6]:
g.payload

OrderedDict([('a',
              [12,
               13,
               omnibelt.nodes.LocalNode._empty_value,
               [],
               omnibelt.nodes.LocalNode._empty_value,
               omnibelt.nodes.LocalNode._empty_value,
               20]),
             ('b', OrderedDict([('x', 1), ('y', 2), ('z', 100)]))])

In [7]:
f = dict(g.flatten())
f

{'a.0': 12, 'a.1': 13, 'a.3': [], 'a.6': 20, 'b.x': 1, 'b.y': 2, 'b.z': 100}

In [8]:
g2 = Node.from_raw(f)
g2.payload

OrderedDict([('a', OrderedDict([('0', 12), ('1', 13), ('3', []), ('6', 20)])),
             ('b', OrderedDict([('x', 1), ('y', 2), ('z', 100)]))])

In [9]:
g2.convert_to_dense('a')
g2.payload

OrderedDict([('a',
              [12,
               13,
               omnibelt.nodes.LocalNode._empty_value,
               [],
               omnibelt.nodes.LocalNode._empty_value,
               omnibelt.nodes.LocalNode._empty_value,
               20]),
             ('b', OrderedDict([('x', 1), ('y', 2), ('z', 100)]))])

In [11]:
g2.get('a.0').parent.payload

[12,
 13,
 omnibelt.nodes.LocalNode._empty_value,
 [],
 omnibelt.nodes.LocalNode._empty_value,
 omnibelt.nodes.LocalNode._empty_value,
 20]

In [ ]:
from omnibelt import capturable_super, captured_super
from omnibelt.tricks import Capturable, captured

In [ ]:
class A(Capturable):
    def captured_super_call(self, src, name, args, kwargs):
        print('!! captured super', self, src, name, args, kwargs)
        return super().captured_super_call(src, name, args, kwargs)

    def captured_method_call(self, src, fn, args, kwargs):
        print('!! captured method', self, src, fn, args, kwargs)
        return super().captured_method_call(src, fn, args, kwargs)

    def f(self, a=1):
        print('A.f', self, a)
        # print(super().f())

class B(A):
    x = -10

    def f(self, a=2):
        print('B.f', self, a)
        super().f()

class C(B):
    @captured
    def f(self, a=3):
        print('C.f', self, a)
        super().f()

class D(C):
    def f(self, a=4):
        print('D.f', self, a)
        super().f()

class E(D):
    @captured
    def f(self, a=5):
        print('E.f', self, a)
        super().f()

class F(E):
    def f(self, a=6):
        print('F.f', self, a)
        super().f()

class G(F):
    pass

In [ ]:
A().f()
print()
B().f()
print()
C().f()
print()
D().f()
print()
E().f()
print()
F().f()
print()
G().f()

In [ ]:
c.f.__closure__[0].cell_contents

In [ ]:
super(C, c).f

In [ ]:
Foo.__init__

In [ ]:
Foo()

In [1]:
from omnidata.framework import hparam, Parameterized

In [5]:
class agnostic:
    def __init__(self, fn, **kwargs):
        super().__init__(**kwargs)
        self.fn = fn

    def __get__(self, instance, owner=None):
        assert owner is not None
        if instance is None:
            instance = owner
        return self.fn.__get__(instance, owner)

In [ ]:
class agnosticproperty:
    def __init__(self, fget, fset=None):
        self.fget = fget
        self.fset = fset

    def __get__(self, obj, owner=None):
        if owner is None:
            owner = type(obj)
        return self.fget.__get__(obj, owner)()

    def __set__(self, obj, value):
        if not self.fset:
            raise AttributeError("can't set attribute")
        type_ = type(obj)
        return self.fset.__get__(obj, type_)(value)

    def setter(self, func):
        if not isinstance(func, agnosticmethod):
            func = agnosticmethod(func)
        self.fset = func
        return self

In [6]:
class A(Parameterized):

    @agnostic
    def f(*args, **kwargs):
        print('A.f', args, kwargs)

    @agnostic
    @property
    def x(*args, **kwargs):
        print('A.x', args, kwargs)
        return 1 + 1


In [11]:
base = A

base.x, base.f(), base().x, base().f();

A.x (<class '__main__.A'>,) {}
A.f (<class '__main__.A'>,) {}
A.x (<__main__.A object at 0x000001A8024E3BB0>,) {}
A.f (<__main__.A object at 0x000001A8024E3BB0>,) {}


In [10]:
base.f, base().f

(<bound method A.f of <class '__main__.A'>>,
 <bound method A.f of <__main__.A object at 0x000001A8024EB5B0>>)